In [26]:
import json
import os

import openai

from data_adaptor import DataAdaptor
from data_loaders import load_2WikiMultihopQA

# Generate Data

In [4]:
wiki_sample = load_2WikiMultihopQA(n_examples=5, split='train')

In [5]:
wiki_adaptor = DataAdaptor(dataset="2WikiMultihopQA")
wiki_examplars = wiki_adaptor.generate_examplars(wiki_sample, strategy="self-ask")
for examplar in wiki_examplars:
    print(examplar)

Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here: Yes.
Follow up: What is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg
Follow up: What is the director of Méditerranée (1963 film)?
Intermediate answer: Jean-Daniel Pollet
Follow up: What is the country of citizenship of Stuart Rosenberg?
Intermediate answer: American
Follow up: What is the country of citizenship of Jean-Daniel Pollet?
Intermediate answer: French
So the final answer is: no

Question: Do both films The Falcon (Film) and Valentin The Good have the directors from the same country?
Are follow up questions needed here: Yes.
Follow up: What is the director of The Falcon (film)?
Intermediate answer: Vatroslav Mimica
Follow up: What is the director of Valentin the Good?
Intermediate answer: Martin Frič
Follow up: What is the country of citizenship of Vatroslav Mimica?
Intermediate answer: Croatian
Fol

In [13]:
wiki_training_examples = wiki_adaptor.generate_training_examples(wiki_sample, strategy="self-ask")
for training_example in wiki_training_examples:
    print(json.dumps(training_example, indent=4))

{
    "prompt": "Question: Are director of film Move (1970 Film) and director of film M\u00e9diterran\u00e9e (1963 Film) from the same country?\nAre follow up questions needed here:\n",
    "target": "Yes.\nFollow up: What is the director of Move (1970 film)?\nIntermediate answer: Stuart Rosenberg\nFollow up: What is the director of M\u00e9diterran\u00e9e (1963 film)?\nIntermediate answer: Jean-Daniel Pollet\nFollow up: What is the country of citizenship of Stuart Rosenberg?\nIntermediate answer: American\nFollow up: What is the country of citizenship of Jean-Daniel Pollet?\nIntermediate answer: French\nSo the final answer is: no\n"
}
{
    "prompt": "Question: Do both films The Falcon (Film) and Valentin The Good have the directors from the same country?\nAre follow up questions needed here:\n",
    "target": "Yes.\nFollow up: What is the director of The Falcon (film)?\nIntermediate answer: Vatroslav Mimica\nFollow up: What is the director of Valentin the Good?\nIntermediate answer: M

In [14]:
# TODO augment prompt with examples

# Format for Finetuning

In [15]:
for training_example in wiki_training_examples:
    # each prompt and completion should and with a simple seperator sequence
    training_example['prompt'] = training_example['prompt'] + '\n\n###\n\n'
    training_example['completion'] = ' ' + training_example.pop('target') + ' END'
    print(len(training_example['prompt'] + training_example['completion'])) # no more than 2048 tokens
for training_example in wiki_training_examples:
    print(json.dumps(training_example, indent=4))

584
651
539
334
376
{
    "prompt": "Question: Are director of film Move (1970 Film) and director of film M\u00e9diterran\u00e9e (1963 Film) from the same country?\nAre follow up questions needed here:\n\n\n###\n\n",
    "completion": " Yes.\nFollow up: What is the director of Move (1970 film)?\nIntermediate answer: Stuart Rosenberg\nFollow up: What is the director of M\u00e9diterran\u00e9e (1963 film)?\nIntermediate answer: Jean-Daniel Pollet\nFollow up: What is the country of citizenship of Stuart Rosenberg?\nIntermediate answer: American\nFollow up: What is the country of citizenship of Jean-Daniel Pollet?\nIntermediate answer: French\nSo the final answer is: no\n END"
}
{
    "prompt": "Question: Do both films The Falcon (Film) and Valentin The Good have the directors from the same country?\nAre follow up questions needed here:\n\n\n###\n\n",
    "completion": " Yes.\nFollow up: What is the director of The Falcon (film)?\nIntermediate answer: Vatroslav Mimica\nFollow up: What is th

In [9]:
openai_sample_filename = "data/2WikiMultihopQA/opeanai_api_sample.jsonl"
with open(openai_sample_filename, "w") as outfile:
    for training_example in wiki_training_examples:
        outfile.write(json.dumps(training_example) + '\n')
    # json.dump(wiki_training_examples, outfile)

In [10]:
# %openai tools fine_tunes.prepare_data -f $openai_sample_filename

In [11]:
os.system(f"openai tools fine_tunes.prepare_data -f {openai_sample_filename}")

Analyzing...

- Your file contains 5 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix `?\nAre follow up questions needed here:\n\n\n###\n\n`. This suffix seems very long. Consider replacing with a shorter suffix, such as `\n\n===\n\n`
- All prompts start with prefix `Question: `
- All completions start with prefix ` Yes.
Follow up: What is the `. Most of the time you should only add the output data into the completion, without any prefix
- All completions end with suffix `\n\n###`

Based on the analysis we will perform the following actions:
- [Recommended] Remove prefix ` Yes.
Follow up: What is the ` from all completions [Y/n]: 

Traceback (most recent call last):
  File "/Users/adamweinberger/opt/anaconda3/envs/compositional_reasoning_finetuning/bin/openai", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/Users/adamweinberger/opt/anaconda3/envs/compositional_reasoning_finetuning/lib/python3.11/site-packages/openai/_openai_scripts.py", line 78, in main
    args.func(args)
  File "/Users/adamweinberger/opt/anaconda3/envs/compositional_reasoning_finetuning/lib/python3.11/site-packages/openai/cli.py", line 599, in prepare_data
    apply_validators(
  File "/Users/adamweinberger/opt/anaconda3/envs/compositional_reasoning_finetuning/lib/python3.11/site-packages/openai/validators.py", line 843, in apply_validators
    df, optional_applied = apply_optional_remediation(
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/adamweinberger/opt/anaconda3/envs/compositional_reasoning_finetuning/lib/python3.11/site-packages/openai/validators.py", line 610, in apply_optional_remediation

256

# Send to OpenAI API

In [19]:
with open("openai_api_key.txt", "r") as file:
    OPENAI_API_KEY = file.read()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [21]:
model_name = "ada"
model_suffix = "crf sample"
os.system(f'openai api fine_tunes.create -t {openai_sample_filename} -m {model_name} --suffix "{model_suffix}"')

Found potentially duplicated files with name 'opeanai_api_sample.jsonl', purpose 'fine-tune' and size 2778 bytes
file-pV0Um8KhhJssuEv5e3CcGQD7
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: 

Upload progress: 100%|██████████| 2.78k/2.78k [00:00<00:00, 2.20Mit/s]


Uploaded file from data/2WikiMultihopQA/opeanai_api_sample.jsonl: file-4pDAOsKZ6VxM6yB0bB8JsHJ1
Created fine-tune: ft-Esqvkl6A08VpsxDOmFVKmtNA
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-06-29 16:01:43] Created fine-tune: ft-Esqvkl6A08VpsxDOmFVKmtNA

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-Esqvkl6A08VpsxDOmFVKmtNA



0

In [23]:
job_id = 'ft-Esqvkl6A08VpsxDOmFVKmtNA'

In [29]:
# status
os.system(f'openai api fine_tunes.follow -i {job_id}')

[2023-06-29 16:01:43] Created fine-tune: ft-Esqvkl6A08VpsxDOmFVKmtNA
[2023-06-29 17:23:06] Fine-tune costs $0.00
[2023-06-29 17:23:06] Fine-tune enqueued. Queue number: 1
[2023-06-29 17:23:12] Fine-tune is in the queue. Queue number: 0
[2023-06-29 17:23:24] Fine-tune started
[2023-06-29 17:23:38] Completed epoch 1/4
[2023-06-29 17:23:39] Completed epoch 2/4
[2023-06-29 17:23:40] Completed epoch 3/4
[2023-06-29 17:23:41] Completed epoch 4/4
[2023-06-29 17:24:06] Uploaded model: ada:ft-university-of-california-berkeley:crf-sample-2023-06-29-21-24-05
[2023-06-29 17:24:11] Uploaded result file: file-gKWXNyMeNxkKyqQ3gjWZHMwj
[2023-06-29 17:24:11] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m ada:ft-university-of-california-berkeley:crf-sample-2023-06-29-21-24-05 -p <YOUR_PROMPT>


0

In [28]:
# results
# model: ada:ft-university-of-california-berkeley:crf-sample-2023-06-29-21-24-05
os.system(f'openai api fine_tunes.results -i {job_id}')

Error: No results file available for fine-tune ft-Esqvkl6A08VpsxDOmFVKmtNA


256

# Use Model

In [ ]:
FINE_TUNED_MODEL = ''
YOUR_PROMPT = ''
openai.Completion.create(
    model=FINE_TUNED_MODEL,
    prompt=YOUR_PROMPT)